# Trip Mapping - Hotel Search

### Dependencies and data

In [1]:
# Dependencies
import os
import requests
import gmaps
import numpy as np
import pandas as pd
from config import GMAPS_KEY

In [2]:
# Configure API key
gmaps.configure(api_key=GMAPS_KEY)

In [3]:
# Weather data
weather_path = os.path.join('data', 'weather.csv')
weather_df = pd.read_csv(weather_path)
weather_df.head(2)

,Id,City,Country,Latitude,Longitude,Time,Description,Max Temp,Humidity,Wind Speed,Cloudiness,Rain,Snow
0,0,Garoowe,SO,8.4054,48.4845,2021-04-18 15:28:30,clear sky,89.92,30,16.42,6,0,0
1,1,Vendas Novas,PT,38.6771,-8.4579,2021-04-18 15:28:30,few clouds,73.40,46,12.66,20,0,0


### Find hotels near destinations within the user's preferred weather

In [4]:
# Lowest and highest `Max Temp` in the data
min_mt, max_mt = weather_df['Max Temp'].min(), weather_df['Max Temp'].max()
min_mt, max_mt

(-17.61, 107.6)

In [5]:
def user_input(prompt, min_temp=min_mt, max_temp=max_mt, binary=False):
    
    """
    Take user input for weather preferences.
    
    Parameters
    ----------
    prompt : str
        Input prompt
    min_temp : float, optional
        Minimum temperature in data, by default {min_mt}
    max_temp : float, optional
        Maximum temperature in data, by default {max_mt}
    binary : bool, optional
        Whether the input should be "yes" or "no" only, by default False
        
    Returns
    -------
    float
        User input
    """
    
    # Init user input and loop condition to False
    i = b = False
    
    # Get user input
    if binary: # yes/no input
        while i not in ['yes', 'no']: # continue until input is either "yes" or "no"
            i = input(prompt) # user input
        i = 1 if i == 'yes' else 0 # convert input to num
    else: 
        prompt += f'\tEnter a number between {min_temp} and {max_temp}: ' 
        while not b: # loop condition
            try:
                i = float(input(prompt)) # get user input as num
            except:
                continue
            if min_temp <= i <= max_temp: # check if user input is within proper temperature range
                b = True # end loop
            
    # Numeric input
    return i
        
    
# Get preferred weather
min_temp = user_input('1. What is the lowest temperature you would like at your destination?\n')
max_temp = user_input('2. What is the highest temperature you would like at your destination?\n', min_temp=min_temp)
rain = user_input('3. Would you like it to rain at your destination? (yes/no) ', binary=True)
snow = user_input('4. Would you like it to snow at your destination? (yes/no) ', binary=True)
min_temp, max_temp, rain, snow # enter 70, 80, no, no

1. What is the lowest temperature you would like at your destination?
	Enter a number between -17.61 and 107.6: 70
2. What is the highest temperature you would like at your destination?
	Enter a number between 70.0 and 107.6: 80
3. Would you like it to rain at your destination? (yes/no) no
4. Would you like it to snow at your destination? (yes/no) no


(70.0, 80.0, 0, 0)

In [6]:
# Filter for cities with the temp preference
pref_cities = weather_df.loc[(weather_df['Max Temp'] >= min_temp) & 
                             (weather_df['Max Temp'] <= max_temp)]

# Filter rain preference
if rain:
    pref_cities = pref_cities.query('Rain > 0')
else:
    pref_cities = pref_cities.query('Rain == 0')

# Filter snow preference
if snow:
    pref_cities = pref_cities.query('Snow > 0')
else:
    pref_cities = pref_cities.query('Snow == 0')
    
pref_cities.shape

(214, 13)

In [7]:
# New df for hotels
hotel_df = pref_cities.reset_index(drop=True)
hotel_df['Hotel'] = np.NaN # new col for hotels

# Base url and params for GMaps Places API
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
payload = {
    'key': GMAPS_KEY,
    'type': 'lodging', # hotels
    'radius': 5000 # meters
}

hotel_df.head(2)

,Id,City,Country,Latitude,Longitude,Time,Description,Max Temp,Humidity,Wind Speed,Cloudiness,Rain,Snow,Hotel
0,1,Vendas Novas,PT,38.6771,-8.4579,2021-04-18 15:28:30,few clouds,73.4,46,12.66,20,0,0,NaN
1,4,Cap Malheureux,MU,-19.9842,57.6142,2021-04-18 15:28:32,overcast clouds,77.0,84,3.00,100,0,0,NaN


In [8]:
# Sample hotel response
payload['location'] = f"{hotel_df.loc[0, 'Latitude']},{hotel_df.loc[0, 'Longitude']}" # 1st city
response = requests.get(base_url, params=payload)
response.json()['results'][0]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 38.678971, 'lng': -8.4540926},
  'viewport': {'northeast': {'lat': 38.68035083029149,
    'lng': -8.452834669708498},
   'southwest': {'lat': 38.6776528697085, 'lng': -8.455532630291502}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
 'name': 'Hotel Acez',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 308,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/106452238628115940643">Hotel Acez</a>'],
   'photo_reference': 'ATtYBwLxSYI6Jl_XDGHk99HPEwXn2MdTjxisBPixYbfS8glsu8jru07i_qefmXaJVh2y5lYkWOhuuqfCD6gahO3KFtWjaZlpd83aW2Xoia7aTIc-MN4gX1dMC4HiPGlzUTjsNvQ4uvHjYY0Aw2dO-ddbdyZ7Y2D_CQYerqa5fYFNGyO_VJ9q',
   'width': 396}],
 'place_id': 'ChIJY6477IifGQ0RgEEsyPVyccQ',
 'plus_code': {'compound_code': 'MGHW+H9 Vendas Novas, Portugal',
  'global_code': '8CCHMGHW+H9'},
 'rating': 4.1,
 'reference': 'ChIJY6477IifGQ0RgEEsyPVyccQ',
 'scope': 'GOOGLE',
 't

In [9]:
# Indices of cities without results
hotel404 = []
print('Starting hotel requests...')

# Find nearest hotel for each city
for idx, row in hotel_df.iterrows():
    
    # Print progress
    if (idx > 0 and idx % 50 == 0) or idx == hotel_df.shape[0] - 1:
        print(f'Requested hotels for {idx} locations.')
        
    # Request hotels
    payload['location'] = f'{row["Latitude"]}, {row["Longitude"]}' # city coordinates
    try:
        response = requests.get(base_url, params=payload).json()
        hotel_df.loc[idx, 'Hotel'] = response['results'][0]['name']
    except:
        hotel404.append(idx)
        
print('Hotel requests complete.\n')
print('Hotels were not found for the following cities:')
print(len(hotel404), hotel_df.loc[hotel404, 'City'].tolist())

Starting hotel requests...
Requested hotels for 50 locations.
Requested hotels for 100 locations.
Requested hotels for 150 locations.
Requested hotels for 200 locations.
Requested hotels for 213 locations.
Hotel requests complete.

Hotels were not found for the following cities:
18 ['Farah', 'Kisanga', 'Issenye', 'Aripuanã', 'Mitú', 'Dutlwe', 'Abadan', 'Tadine', 'Amapá', 'Bondo', 'Maneromango', 'Gandajika', 'Micomeseng', 'Nālūt', 'Micheweni', 'Aketi', 'Caconda', 'Poya']


In [10]:
# Drop cities without hotel results
hotel_df.dropna(inplace=True)
hotel_df.shape

(194, 14)

### Temperature heatmap

In [11]:
# For Jupyterlab
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# !jupyter lab build

In [14]:
# Locations and measurements
locations = hotel_df[['Latitude', 'Longitude']]
weights = hotel_df['Max Temp'].apply(lambda t: max(t, 0))

# Popup info box
hotel_info = """
<dl>
<dt>Hotel</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}, {Country}</dd>
<dt>Weather</dt><dd>{Max Temp} F - {Description}</dd>
</dl>
"""

# Layers
heatmap = gmaps.heatmap_layer(locations, weights, dissipating=False, point_radius=3, max_intensity=300)
markers = gmaps.marker_layer(locations,
                             info_box_content=[hotel_info.format(**row) for _, row in hotel_df.iterrows()])

# Heatmap
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=1.5)
fig.add_layer(heatmap)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Save hotel data
hotel_path = os.path.join('data', 'hotels.csv')
hotel_df.to_csv(hotel_path, index=False)
pd.read_csv(hotel_path).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           194 non-null    int64  
 1   City         194 non-null    object 
 2   Country      194 non-null    object 
 3   Latitude     194 non-null    float64
 4   Longitude    194 non-null    float64
 5   Time         194 non-null    object 
 6   Description  194 non-null    object 
 7   Max Temp     194 non-null    float64
 8   Humidity     194 non-null    int64  
 9   Wind Speed   194 non-null    float64
 10  Cloudiness   194 non-null    int64  
 11  Rain         194 non-null    int64  
 12  Snow         194 non-null    int64  
 13  Hotel        194 non-null    object 
dtypes: float64(4), int64(5), object(5)
memory usage: 21.3+ KB
